In [9]:
import pandas as pd
import numpy as np
from sklearn import svm, neighbors,tree, ensemble, model_selection, preprocessing
from sklearn.model_selection import train_test_split


In [10]:
df = pd.read_csv('fulldata_colonnes_essentielles.csv')

In [11]:
# Les lignes ayant le même 'Accession' doivent être comprises comme les données d'une même entité (code Amal)
np.random.seed = 123

unique_users = df['ACCESSION'].unique()
target_n = int(.8 * len(df))
shuffled_users = np.random.permutation(unique_users)

user_count = df['ACCESSION'].value_counts()

mapping = user_count.reindex(shuffled_users).cumsum() <= target_n
mask = df['ACCESSION'].map(mapping)

df_train = df[mask]
df_test = df[~mask]

df_train.head(4)

,Unnamed: 0,ACCESSION,class1,class2,class3,AC_mode,ion_precursor_m/z,num_peak,peak_m/z,peak_rel.int.
14,61,PM000301,Phenylpropanoid,Phenylpropanoid ester,Quinic acid,1,353.0,4,191.0,100
15,62,PM000301,Phenylpropanoid,Phenylpropanoid ester,Quinic acid,1,353.0,4,179.0,76
16,63,PM000301,Phenylpropanoid,Phenylpropanoid ester,Quinic acid,1,353.0,4,135.0,19
17,64,PM000301,Phenylpropanoid,Phenylpropanoid ester,Quinic acid,1,353.0,4,161.0,10


In [12]:
#Séparation en variables cibles et explicatives :  chaque classe est une variable cible différente.
#La classe la plus interessante a priori est la classe 3, la plus 'précise' (classe 3 incluse dans la 2 qui est incluse dans la 1)
 
X_test = df_test[['ion_precursor_m/z', 'num_peak', 'peak_m/z', 'peak_rel.int.', 'AC_mode' ]]
y_test3 = df_test.class3
y_test2 = df_test.class2
y_test1 = df_test.class1

X_train = df_train[['ion_precursor_m/z', 'num_peak', 'peak_m/z', 'peak_rel.int.', 'AC_mode']]
y_train3 = df_train.class3
y_train2 = df_train.class2
y_train1 = df_train.class1

In [13]:
#classification paramètre par défaut rf CLASS3
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train3)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test3))

Score de rf: 0.4849708389412293


In [14]:
#classification paramètre par défaut rf CLASS2
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train2)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test2))

Score de rf: 0.55697622252131


In [15]:
#classification paramètre par défaut rf CLASS1
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train1)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test1))

Score de rf: 0.6867429340511441


In [7]:
#recherche des meilleurs paramètres forêt aléatoire CLASS3

clf_rf = ensemble.RandomForestClassifier()

parametres_rf = {}

grid_clf_rf = model_selection.GridSearchCV(estimator = clf_rf, param_grid = parametres_rf)

grille = grid_clf_rf.fit(X_train, y_train3)

print('Meilleurs paramètres de rf:', grid_clf_rf.best_params_)

y_pred_rf = grid_clf_rf.predict(X_test)

print('Score de rf:', grid_clf_rf.score(X_test, y_test3))